# Bagging para la regresión

En este notebook vamos a emplear técnicas de bagging para predecir el número de likes de una publicación. Empleamos de nuevo los datos ya limpios extraídos en los notebook de la tercera sección.

Los módulos a emplear son:

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

Cargamos los datos a partir del csv generado en la tercera sección:

In [ ]:
data_fb = pd.read_csv('./data/cleaned_facebook.csv')
data_fb.head()

Separamos la variable objetivo (like) del resto de variables que actuarán como predictoras:

In [ ]:
X = data_fb.drop('like', axis=1)
y = data_fb['like']

Dividimos los datos en datos de entrenamiento y datos de validación con el fin de detectar el overfitting y generar una medida objetiva de la calidad del modelo que nos permita compararlo con los ya calculados hasta ahora:

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21)

## Construcción del modelo

A continuación procedemos a generar nuestro modelo de bagging. Si recordamos los modelos de bagging se construyen como una unión de árboles por lo que instanciamos en primer lugar el tipo de árbol básico; un árbol de regresión como el que vimos en notebook previos:

In [ ]:
from sklearn.tree import DecisionTreeRegressor 
base_reg = DecisionTreeRegressor() 

Comenzamos como siempre instanciando el modelo al que le pasamos como parámetro el tipo de modelo base que vamos a utilizar:

In [ ]:
from sklearn.ensemble import BaggingRegressor
regressorBag = BaggingRegressor(base_estimator = base_reg)

Una vez instanciado el modelo procedemos como ya es habitual a ajustarlo a los datos:

In [ ]:
regressorBag.fit(X_train, y_train)

Podemos observar el número de estimadores (de árboles) empleados así como las máximas muestras permitidas en un nodo:

In [ ]:
regressorBag.n_estimators

In [ ]:
regressorBag.max_samples

Una vez hecho esto ya tenemos nuestro modelo entrenado, podemos proceder como siempre al cálculo del R2 en los datos de entrenamiento y validación para medir la calidad y detectar el sobreajuste de haberlo:

In [ ]:
y_bag_pred_train = regressorBag.predict(X_train)

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_train, y_bag_pred_train)

In [ ]:
y_bag_pred_test = regressorBag.predict(X_test)

In [ ]:
r2_score(y_test, y_bag_pred_test)

Detectamos un overfitting bastante alto. Vamos a jugar con algunos parámetros para intentar mejorar nuestro modelo:

In [ ]:
num_trees = 1000
max_samples = 8

In [ ]:
regressorBag_tuned = BaggingRegressor(base_estimator = base_reg, n_estimators = num_trees, 
                                      max_samples= max_samples, random_state=21)
regressorBag_tuned.fit(X_train, y_train)

In [ ]:
y_bag_tuned_pred_train = regressorBag_tuned.predict(X_train)
r2_score(y_train, y_bag_tuned_pred_train)

In [ ]:
y_bag_tuned_pred_test = regressorBag_tuned.predict(X_test)
r2_score(y_test, y_bag_tuned_pred_test)

Mitigado el problema del overfitting procedemos al cálculo del error medio cuadrático del bagging:

In [ ]:
from sklearn.metrics import mean_squared_error
mse_test = mean_squared_error(y_test, y_bag_tuned_pred_test)
np.sqrt(mse_test)

Por último el bagging nos permite observar que variables han sido las más importantes a la hora de construir nuestras predicciones, las observamos agrupadas en un dataframe:

In [ ]:
feature_importances = np.mean([
    tree.feature_importances_ for tree in regressorBag_tuned.estimators_
], axis=0)

In [ ]:
i = 0
feature_importance_dic = {}
for element in feature_importances:
    feature_importance_dic[X_train.columns[i]] = element
    i = i + 1
importance_df = pd.DataFrame.from_dict(feature_importance_dic, orient='index', columns=['Importancia'])
importance_df.sort_values('Importancia', ascending=False)

Observamos que el número total de likes es sin duda la variable más indicativa para este modelo.